In [ ]:
import pygsheets
from google.cloud import secretmanager
from google.cloud import bigquery
from datetime import datetime 
from datetime import timedelta

import requests
import os
import urllib
import google.oauth2.id_token
import google.auth.transport.requests
import subprocess
import pygsheets
from google.cloud import secretmanager

from credentials import PROJECT_ID, SM_SECRET_NAME, SHEET_KEY_HEIRLOOM_PRICING_MODEL, SHEET_TAB_COMPLIANCE_TOGGLE, CF_GET_TOKEN_V2_URL

In [ ]:
# ------ SECRET MANAGER ---------
class SecretManager():
    """ Construct Secret Manager client
    """
    client = secretmanager.SecretManagerServiceClient()

    def __init__(self, project_id):
        self.project_id = project_id
        self.parent = f"projects/{project_id}"

    def get_secret(self, secret_id, version_id:str = "latest"):
        """ Get secret value

            TODO:
                - Raise if secret not found
            Args:
                secret_id (str): Secret ID
                version_id (str): Secret version ID
                    default: latest
            Returns:
                secret_value (str): Secret value
        """
        # Access the secret version
        response = self.client.access_secret_version(
            request={
                "name": f"projects/{self.project_id}/secrets/{secret_id}/versions/{version_id}"
                })
        print("Accessed secret version: {}".format(response.name))
        # Return the decoded payload
        return response.payload.data.decode("UTF-8")     

### Fetching data from Google Sheets

In [ ]:
# Get the SA key from Secret Manager
sm = SecretManager(PROJECT_ID)
sa_key_json = sm.get_secret(SM_SECRET_NAME)

In [ ]:
gc = pygsheets.authorize(service_account_json=sa_key_json)
sheet_pricingModel = gc.open_by_key(SHEET_KEY_HEIRLOOM_PRICING_MODEL)
sheet_complianceToggle = sheet_pricingModel.worksheet_by_title(SHEET_TAB_COMPLIANCE_TOGGLE)
df_complianceToggle = sheet_complianceToggle.get_as_df()
df_complianceToggle

### Fetching data from BigQuery

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

In [ ]:
query = bq_client.query("SELECT * FROM stayloom.Listings.t_listing_dictionary")
df_listing_dict = query.result().to_dataframe() 
df_listing_dict

### Analysis to Join data from both resources

In [ ]:
# renaming column name in sheets df for join
df_complianceToggle.rename(columns={'Listing Unit Address': 'Listing_Unit_Address'}, inplace=True)

In [ ]:
# listing_unit_address is the common column in both df
df_listingDict_listedTrue = df_listing_dict[['External_ID', 'Listing_Unit_Address']][df_listing_dict['Listied_on_any_channels_'] == True] # filtering the listings which are listed to get unique external id
df_listingDict_listedTrue

### Joining both df

In [ ]:
df_joined = df_complianceToggle.merge(df_listingDict_listedTrue[['External_ID', 'Listing_Unit_Address']], on='Listing_Unit_Address')
df_joined

### time zone mapping

|ABBREVIATION	|TIME ZONE NAME	                |UTC OFFSET
|---------------|-------------------------------|----------
|AST	        |ATLANTIC STANDARD TIME	        |UTC - 4
|EST	        |EASTERN STANDARD TIME	        |UTC - 5
|EDT	        |EASTERN DAYLIGHT TIME	        |UTC - 4
|CST	        |CENTRAL STANDARD TIME	        |UTC - 6
|CDT	        |CENTRAL DAYLIGHT TIME	        |UTC - 5
|MST	        |MOUNTAIN STANDARD TIME	        |UTC - 7
|MDT	        |MOUNTAIN DAYLIGHT TIME	        |UTC - 6
|PST	        |PACIFIC STANDARD TIME	        |UTC - 8
|PDT	        |PACIFIC DAYLIGHT TIME	        |UTC - 7
|AKST	        |ALASKA TIME	                |UTC - 9
|AKDT	        |ALASKA DAYLIGHT TIME	        |UTC - 8
|HST	        |HAWAII STANDARD TIME	        |UTC - 10
|HAST	        |HAWAII-ALEUTIAN STANDARD TIME	|UTC - 10
|HADT	        |HAWAII-ALEUTIAN DAYLIGHT TIME	|UTC - 9
|SST	        |SAMOA STANDARD TIME	        |UTC - 11
|SDT	        |SAMOA DAYLIGHT TIME	        |UTC - 10
|CHST	        |CHAMORRO STANDARD TIME	        |UTC +10

In [ ]:
utc_now = datetime.utcnow() 
mapping_timezone = {
    'AST': (utc_now - timedelta(hours=4)).strftime('%H:%M'),
    'EST': (utc_now - timedelta(hours=5)).strftime('%H:%M'),
    'EDT': (utc_now - timedelta(hours=4)).strftime('%H:%M'),
    'CST': (utc_now - timedelta(hours=6)).strftime('%H:%M'),
    'CDT': (utc_now - timedelta(hours=5)).strftime('%H:%M'),
    'MST': (utc_now - timedelta(hours=7)).strftime('%H:%M'),
    'MDT': (utc_now - timedelta(hours=6)).strftime('%H:%M'),
    'PST': (utc_now - timedelta(hours=8)).strftime('%H:%M'),
    'PDT': (utc_now - timedelta(hours=7)).strftime('%H:%M'),
    'AKST': (utc_now - timedelta(hours=9)).strftime('%H:%M'),
    'AKDT': (utc_now - timedelta(hours=8)).strftime('%H:%M'),
    'HST': (utc_now - timedelta(hours=10)).strftime('%H:%M'),
    'HAST': (utc_now - timedelta(hours=10)).strftime('%H:%M'),
    'HADT': (utc_now - timedelta(hours=9)).strftime('%H:%M'),
    'SST': (utc_now - timedelta(hours=11)).strftime('%H:%M'),
    'SDT': (utc_now - timedelta(hours=10)).strftime('%H:%M'),
    'CHST': (utc_now + timedelta(hours=10)).strftime('%H:%M')
}
mapping_timezone

### populating actual occupancy for each listing

In [ ]:
df_joined['actual_occupancy'] = ''
df_joined

In [ ]:
# function to apply logic of populating actual occupancy 
def set_actual_occupancy(df_row):
    timezone = df_row['Timezone'] # fetching timezone of a listing
    tz_time = mapping_timezone[timezone] # mapping timezone with its current time 
    time_in = df_row['Time In']
    time_out = df_row['Time Out']
    
    # if the current time of the listing's timezone lies between time in and time out value of that listing then 
    # Number of Occupants (In) value will be actual occupancy value
    if datetime.strptime(time_in, '%H:%M') <= datetime.strptime(tz_time, '%H:%M') <= datetime.strptime(time_out, '%H:%M'):
        return df_row['Number of Occupants (In)']
    else:
        return df_row['Number of Occupants (Out)']

In [ ]:
df_joined['actual_occupancy'] = df_joined.apply(set_actual_occupancy, axis= 1)
df_joined

### applying actual occupancy to Guesty listings

In [ ]:
# Get the Guesty API key
def get_token(url):
    """ Generate token and make request to endpoint

        Docs:
            - https://cloud.google.com/functions/docs/securing/authenticating#console
    """
    # Environment: C.Functions
    if "FUNCTION_TARGET" in os.environ:
        req = urllib.request.Request(url)
        auth_req = google.auth.transport.requests.Request()
        id_token = google.oauth2.id_token.fetch_id_token(auth_req, url)
        req.add_header("Authorization", f"Bearer {id_token}")
        response = urllib.request.urlopen(req)
        return response.read().decode("utf-8")
    # Environment: Local
    else:
        # HACK: Get the token from gcloud
        gcloud_itoken = subprocess.check_output(["gcloud","auth", "print-identity-token"], shell= True)
        token = gcloud_itoken.decode().strip()
        # Make request
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {token}'
        }
        return requests.get(url, headers=headers).text

In [ ]:
token = get_token(CF_GET_TOKEN_V2_URL)
# print(token)

guesty_headers = {
    'Authorization': f'Bearer {token}',
    'content-type': 'application/json'
}

In [ ]:
# # function to populate actual occupancy (custom field) of each listing on Guesty
# def populating_actualOcc_toGuesty(df_row):
#     url_update_actualOcc = f"https://open-api.guesty.com/v1/listings/{df_row['External_ID']}/custom-fields"
#     payload = { "customFields": [
#             {
#                 "fieldId": "5ae01c0b34249a00248b2f62",
#                 "value": str(df_row['actual_occupancy'])
#             }
#         ] }
#     response = requests.put(url_update_actualOcc, json=payload, headers=guesty_headers)

#     print(f"Finished with status code: {response.status_code}")

In [ ]:
# function to populate accommodates (actual field) of each listing on Guesty
def populating_actualOcc_toGuesty(df_row):
    url_update_accomodate = f"https://open-api.guesty.com/v1/listings/{df_row['External_ID']}"
    payload = { "accommodates": str(df_row['actual_occupancy']) }
    response = requests.put(url_update_accomodate, json=payload, headers=guesty_headers)

    print(f"API responded with status code: {response.status_code}")

In [ ]:
df_joined.apply(populating_actualOcc_toGuesty, axis=1)